In [149]:
import os
import re
import operator
import warnings
import time
import numpy as np
import simplejson as json
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn.cluster import SpectralClustering

import logging
import csv
import pickle

import numpy as np


In [150]:
# preprocess the petitions data(can use the dataset to test)

In [203]:
docs=[]
with open('petitions_complete.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            docs.append(row)
from nltk import pos_tag, word_tokenize

for passage in docs:
    passage['body']=[x for x in word_tokenize(passage['body']) if x.isalpha()]
    
                   

In [ ]:
#  make feature function for NamedEntityChunker model, which is necessary

In [141]:
import string
from nltk.stem.snowball import SnowballStemmer
corpus_root = "gmb-2.2.0" 
 
def features(tokens, index, history):
    """
    `tokens`  = a POS-tagged sentence [(w1, t1), ...]
    `index`   = the index of the token we want to extract features for
    `history` = the previous predicted IOB tags
    """
 
    # init the stemmer
    stemmer = SnowballStemmer('english')
 
    # Pad the sequence with placeholders
    tokens = [('[START2]', '[START2]'), ('[START1]', '[START1]')] + list(tokens) + [('[END1]', '[END1]'), ('[END2]', '[END2]')]
    history = ['[START2]', '[START1]'] + list(history)
 
    # shift the index with 2, to accommodate the padding
    index += 2
 
    word, pos = tokens[index]
    prevword, prevpos = tokens[index - 1]
    prevprevword, prevprevpos = tokens[index - 2]
    nextword, nextpos = tokens[index + 1]
    nextnextword, nextnextpos = tokens[index + 2]
    previob = history[index - 1]
    contains_dash = '-' in word
    contains_dot = '.' in word
    allascii = all([True for c in word if c in string.ascii_lowercase])
 
    allcaps = word == word.capitalize()
    capitalized = word[0] in string.ascii_uppercase
 
    prevallcaps = prevword == prevword.capitalize()
    prevcapitalized = prevword[0] in string.ascii_uppercase
 
    nextallcaps = prevword == prevword.capitalize()
    nextcapitalized = prevword[0] in string.ascii_uppercase
 
    return {
        'word': word,
        'lemma': stemmer.stem(word),
        'pos': pos,
        'all-ascii': allascii,
 
        'next-word': nextword,
        'next-lemma': stemmer.stem(nextword),
        'next-pos': nextpos,
 
        'next-next-word': nextnextword,
        'nextnextpos': nextnextpos,
 
        'prev-word': prevword,
        'prev-lemma': stemmer.stem(prevword),
        'prev-pos': prevpos,
 
        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,
 
        'prev-iob': previob,
 
        'contains-dash': contains_dash,
        'contains-dot': contains_dot,
 
        'all-caps': allcaps,
        'capitalized': capitalized,
 
        'prev-all-caps': prevallcaps,
        'prev-capitalized': prevcapitalized,
 
        'next-all-caps': nextallcaps,
        'next-capitalized': nextcapitalized,
    }

In [ ]:
#  Groningen Meaning Bank (GMB) though.
# GMB is a fairly large corpus with a lot of annotations
# import corpus


In [192]:
def to_conll_iob(annotated_sentence):
    """
    `annotated_sentence` = list of triplets [(w1, t1, iob1), ...]
    Transform a pseudo-IOB notation: O, PERSON, PERSON, O, O, LOCATION, O
    to proper IOB notation: O, B-PERSON, I-PERSON, O, O, B-LOCATION, O
    """
    proper_iob_tokens = []
    for idx, annotated_token in enumerate(annotated_sentence):
        tag, word, ner = annotated_token
 
        if ner != 'O':
            if idx == 0:
                ner = "B-" + ner
            elif annotated_sentence[idx - 1][2] == ner:
                ner = "I-" + ner
            else:
                ner = "B-" + ner
        proper_iob_tokens.append((tag, word, ner))
    return proper_iob_tokens
 
 
def read_gmb(corpus_root):
    for root, dirs, files in os.walk(corpus_root):
        for filename in files:
            if filename.endswith(".tags"):
                with open(os.path.join(root, filename), 'rb') as file_handle:
                    file_content = file_handle.read().decode('utf-8').strip()
                    annotated_sentences = file_content.split('\n\n')
                    for annotated_sentence in annotated_sentences:
                        annotated_tokens = [seq for seq in annotated_sentence.split('\n') if seq]
 
                        standard_form_tokens = []
 
                        for idx, annotated_token in enumerate(annotated_tokens):
                            annotations = annotated_token.split('\t')
                            word, tag, ner = annotations[0], annotations[1], annotations[3]
 
                            if ner != 'O':
                                ner = ner.split('-')[0]
 
                            if tag in ('LQU', 'RQU'):   # Make it NLTK compatible
                                tag = "``"
 
                            standard_form_tokens.append((word, tag, ner))
 
                        conll_tokens = to_conll_iob(standard_form_tokens)
 
                        # Make it NLTK Classifier compatible - [(w1, t1, iob1), ...] to [((w1, t1), iob1), ...]
                        # Because the classfier expects a tuple as input, first item input, second the class
                        yield [((w, t), iob) for w, t, iob in conll_tokens]
 


In [193]:
import pickle
from collections import Iterable
from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
 
 
class NamedEntityChunker(ChunkParserI):
    def __init__(self, train_sents, **kwargs):
        assert isinstance(train_sents, Iterable)
 
        self.feature_detector = features
        self.tagger = ClassifierBasedTagger(
            train=train_sents,
            feature_detector=features,
            **kwargs)
 
    def parse(self, tagged_sent):
        chunks = self.tagger.tag(tagged_sent)
 
        # Transform the result from [((w1, t1), iob1), ...] 
        # to the preferred list of triplets format [(w1, t1, iob1), ...]
        iob_triplets = [(w, t, c) for ((w, t), c) in chunks]
        iob_triplets=[(w, t, c) for (w, t, c) in iob_triplets if c !='O']
        # Transform the list of triplets to nltk.Tree format
        return conlltags2tree(iob_triplets)


In [194]:
reader = read_gmb(corpus_root)
data = list(reader)
training_samples = data[:int(len(data) * 0.9)]
test_samples = data[int(len(data) * 0.9):]
print(len(data))
print ("#training samples = %s" % len(training_samples))    # training samples = 55809
print ("#test samples = %s" % len(test_samples))                # test samples = 6201
 
 


62010
#training samples = 55809
#test samples = 6201


In [191]:
data[0:10]

[[(('Thousands', 'NNS'), 'O'),
  (('of', 'IN'), 'O'),
  (('Kenyans', 'NNS'), 'B-gpe'),
  (('demonstrated', 'VBD'), 'O'),
  (('in', 'IN'), 'O'),
  (('Nairobi', 'NNP'), 'B-geo'),
  (('Tuesday', 'NNP'), 'B-tim'),
  (('to', 'TO'), 'O'),
  (('protest', 'VB'), 'O'),
  (('last', 'JJ'), 'O'),
  (('week', 'NN'), 'O'),
  (("'s", 'POS'), 'O'),
  (('police', 'NN'), 'O'),
  (('raids', 'NNS'), 'O'),
  (('on', 'IN'), 'O'),
  (('a', 'DT'), 'O'),
  (('national', 'JJ'), 'O'),
  (('media', 'NNS'), 'O'),
  (('company', 'NN'), 'O'),
  (('.', '.'), 'O')],
 [(('At', 'IN'), 'O'),
  (('least', 'JJS'), 'O'),
  (('2000', 'CD'), 'B-tim'),
  (('people', 'NNS'), 'O'),
  ((',', ','), 'O'),
  (('including', 'VBG'), 'O'),
  (('opposition', 'NN'), 'O'),
  (('lawmakers', 'NNS'), 'O'),
  (('from', 'IN'), 'O'),
  (('the', 'DT'), 'O'),
  (('Orange', 'NNP'), 'B-org'),
  (('Democratic', 'NNP'), 'I-org'),
  (('Movement', 'NNP'), 'I-org'),
  ((',', ','), 'O'),
  (('took', 'VBD'), 'O'),
  (('part', 'NN'), 'O'),
  (('in', 'IN'),

In [195]:
chunker = NamedEntityChunker(training_samples)

In [196]:
# pre function to formate tokenize data and remove stop word

In [197]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
 

def pre(passage):

    passage=[x for x in word_tokenize(passage) if x.isalpha()]
    ps = PorterStemmer()
    passage1=[]
    for word in passage:
        passage1.append(ps.stem(word))
    print(passage1)
    passage1=list(set(passage1))
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    for w in passage1:
        if w not in stopWords:
            wordsFiltered.append(w)
    return(wordsFiltered)

In [198]:
data = "Cohen, who has been under criminal investigation for months, which is separate from the special counsel case, has been rushing to meet U.S. District Court for the Southern District of New York Judge Kimba Wood’s Friday deadline to complete a privilege review of over 3.7 million documents seized in the April 9 raids of Cohen’s New York properties and law office.Representatives from the Southern District of New York declined to comment.Cohen, who is under federal investigation now with no legal representation, is likely to cooperate with federal prosecutors in Manhattan, sources said. This development, which is believed to be imminent, will likely hit the White House, family members, staffers and counsels hard."

In [199]:
# get train rescult

In [200]:
print(chunker.parse(pos_tag(pre(data))))

['cohen', 'who', 'ha', 'been', 'under', 'crimin', 'investig', 'for', 'month', 'which', 'is', 'separ', 'from', 'the', 'special', 'counsel', 'case', 'ha', 'been', 'rush', 'to', 'meet', 'district', 'court', 'for', 'the', 'southern', 'district', 'of', 'new', 'york', 'judg', 'kimba', 'wood', 's', 'friday', 'deadlin', 'to', 'complet', 'a', 'privileg', 'review', 'of', 'over', 'million', 'document', 'seiz', 'in', 'the', 'april', 'raid', 'of', 'cohen', 's', 'new', 'york', 'properti', 'and', 'law', 'from', 'the', 'southern', 'district', 'of', 'new', 'york', 'declin', 'to', 'who', 'is', 'under', 'feder', 'investig', 'now', 'with', 'no', 'legal', 'represent', 'is', 'like', 'to', 'cooper', 'with', 'feder', 'prosecutor', 'in', 'manhattan', 'sourc', 'said', 'thi', 'develop', 'which', 'is', 'believ', 'to', 'be', 'immin', 'will', 'like', 'hit', 'the', 'white', 'hous', 'famili', 'member', 'staffer', 'and', 'counsel', 'hard']
(S
  (org white/JJ)
  (art properti/NN)
  (tim separ/NN cooper/NN ha/NN feder/N

In [201]:
# train the petition dataset

In [204]:
for passage in docs:
    passage['body']=chunker.parse((pos_tag(passage['body'])))

In [205]:
print((docs[1]["body"]))

(S
  (org Keystone/NNP XL/NNP)
  (tim sands/NNS in/IN)
  (geo Alberta/NNP)
  (org Big/NNP Oil/NN)
  (gpe Canadian/NNP)
  (nat Additionally/NNP tar/NN)
  (org tar/NN)
  (org Keystone/NNP XL/NNP permit/NN))


In [206]:
score = chunker.evaluate([conlltags2tree([(w, t, iob) for (w, t), iob in iobs if iob !='O']) for iobs in test_samples[:500]])
print (score.accuracy()  )      # 0.931132334092 - Awesome :D

0.49791790600832836


In [207]:
# only test the accuracy of the dataset iob !="O"( they are the time and location feature),
# the accuracy is not great as the whole dataset